# EJERCICIOS

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier,StackingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_predict,cross_val_score,cross_validate
from sklearn.preprocessing import StandardScaler

In [29]:
warnings.filterwarnings("ignore", category=FutureWarning)

## EJERCICIO 1

Si has entrenado cinco modelos diferentes en el mismo conjunto de entrenamiento exacto y todos consiguen una precisión del 95%, ¿hay alguna posibilidad de que puedas combinar estos modelos para obtener mejores resultados? 

Si la respuesta es sí, ¿cómo? Si la respuesta es no, ¿por qué?

Si es posible combianr estos modelos y puede ser que mejoren los resultados.
Puedes usar varios metodos de ensamblaje.
- Voting
- Stacking
- Bagging

## EJERCICIO 2

Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50.000 instancias para entrenamiento, 10.000 para validación y 10.000 para pruebas). 

Después, entrena varios clasificadores diferentes (uno de ellos que sea un árbol de decisión). 

A continuación, intenta combinarlos en un ensamble que supere en rendimiento a cada clasificador individual del conjunto de validación, utilizando hard voting. 

Una vez que hayas encontrado uno, pruébalo en el conjunto de pruebas.

In [30]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto" )
X, y = mnist.data, mnist.target

In [31]:
## Conjunto de entrenamiento
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=50000, stratify=y, random_state=42)
## Conjunto de prueba(test) y validacion
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, train_size=10000, stratify=y_temp, random_state=42)

In [32]:
models = {
    'Regresión Logística': LogisticRegression(max_iter=10000, random_state=42),
    'Arbol de decisiones': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier()
}

# Bucle para entrenar, evaluar y calcular la matriz de confusión para cada modelo
for name, model in models.items():
    print(f"Evaluando {name}:")
    
    model_pipeline=make_pipeline(StandardScaler(),model)

    model_pipeline.fit(X_train,y_train)
    
    # Calcular exactitud (accuracy) en el conjunto de entrenamiento
    accuracy = accuracy_score(y_val, model_pipeline.predict(X_val))
    print(f"Exactitud en el conjunto de entrenamiento: {accuracy:.4f}\n")   



Evaluando Regresión Logística:
Exactitud en el conjunto de entrenamiento: 0.9107

Evaluando Arbol de decisiones:
Exactitud en el conjunto de entrenamiento: 0.8678

Evaluando Random Forest:
Exactitud en el conjunto de entrenamiento: 0.9670

Evaluando KNeighborsClassifier:
Exactitud en el conjunto de entrenamiento: 0.9422



In [33]:
voting_clf = VotingClassifier(
    estimators=[
        ('reg_log', LogisticRegression(max_iter=5000,random_state=42)),
        ('tree_cla', DecisionTreeClassifier(random_state=42)),
        ('rnd_cl', RandomForestClassifier(random_state=42)),
        ('kn_cl', KNeighborsClassifier())
    ],voting='hard',
    weights=[1, 1, 3, 1]
)

voting_clf_pipeline = make_pipeline(StandardScaler(), voting_clf)

voting_clf_pipeline.fit(X_train,y_train)

# Calcular exactitud del ensemble
accuracy_voting = accuracy_score(y_val,voting_clf_pipeline.predict(X_val))
print(f"Exactitud del ensemble (Hard Voting): {accuracy_voting:.4f}")


Exactitud del ensemble (Hard Voting): 0.9674


## EJERCICIO 3

Ejecuta los clasificadores individuales del ejercicio anterior para hacer predicciones en el conjunto de entrenamiento y crea un nuevo conjunto de entrenamiento con las predicciones resultantes: cada instancia de entrenamiento es un vector que contiene el conjunto de predicciones de todos tus clasificadores para una imagen y el objetivo es la clase de la imagen. Entrena un clasificador (RandomForestClassifier) en este nuevo conjunto de entrenamiento. 

Acabas de entrenar un blender y, junto a los clasificadores, forma un ensamble de stacking.

Ahora, evalúa el ensamble en el conjunto de prueba. 

¿Cómo es en comparación con el clasificador de votación que has entrenado antes?

Haz lo mismo usando StackingClassifier

In [ ]:
models = {
    'Regresión Logística': LogisticRegression(max_iter=10000, random_state=42),
    'Árbol de decisiones': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier()
}

# Crear un array para almacenar las predicciones de cada modelo
base_predictions_train = []
base_predictions_test = []

# Bucle para entrenar cada modelo y generar sus predicciones en el conjunto de entrenamiento
for name, model in models.items():
    print(f"Entrenando y evaluando {name}:")

    # Crear un pipeline con escalado y el modelo
    model_pipeline = make_pipeline(StandardScaler(), model)

    # Entrenar el modelo en el conjunto de entrenamiento
    model_pipeline.fit(X_train, y_train)

    # Generar las predicciones del modelo en el conjunto de entrenamiento
    y_train_pred = model_pipeline.predict(X_train)

    # Agregar las predicciones al array
    base_predictions_train.append(y_train_pred)

# Bucle para entrenar cada modelo y generar sus predicciones en el conjunto de prueba
for name, model in models.items():
    print(f"Entrenando y evaluando {name}:")

    # Crear un pipeline con escalado y el modelo
    model_pipeline = make_pipeline(StandardScaler(), model)

    # Generar las predicciones del modelo en el conjunto de entrenamiento
    y_test_pred = model_pipeline.predict(X_test)

    # Agregar las predicciones al array
    base_predictions_test.append(y_test_pred)

# Convertir las predicciones en un DataFrame
base_predictions_train_df = pd.DataFrame(np.array(base_predictions_train).T, columns=models.keys())  # Transponer y poner nombres a las columnas
base_predictions_test_df = pd.DataFrame(np.array(base_predictions_test).T, columns=models.keys())  # Transponer y poner nombres a las columnas

Entrenando y evaluando Regresión Logística:
Entrenando y evaluando Árbol de decisiones:
Entrenando y evaluando Random Forest:
Entrenando y evaluando KNeighborsClassifier:
Entrenando y evaluando Regresión Logística:
Entrenando y evaluando Árbol de decisiones:
Entrenando y evaluando Random Forest:
Entrenando y evaluando KNeighborsClassifier:


In [36]:
base_predictions_train_df.head()

,Regresión Logística,Árbol de decisiones,Random Forest,KNeighborsClassifier
0,3,3,3,3
1,9,9,9,9
2,1,1,1,1
3,2,2,2,2
4,1,1,1,1


Creamos el conjunto de entrenamiento

In [37]:
new_x_train=base_predictions_train_df
new_y_train= y_train
new_X_test=base_predictions_test_df
new_y_test=y_test

ENTRENAMOS EL META MODELO CON EL NUEVO CONJUNTO DE ENTRENAMIENTO

In [ ]:
rf_meta = RandomForestClassifier(random_state=42)
rf_meta.fit(new_x_train, new_y_train)

# Calcular exactitud del ensemble meta-modelo
accuracy_meta_model = accuracy_score(new_y_test, rf_meta.predict(new_X_test))
print(f"Exactitud del ensemble meta-modelo (RandomForest): {accuracy_meta_model:.4f}")

Exactitud del ensemble meta-modelo (RandomForest): 1.0000


STACKING CLASSIFIER

In [39]:
stacking_clf = StackingClassifier(
    estimators=[
        ('reg_log', LogisticRegression(max_iter=5000,random_state=42)),
        ('tree_cla', DecisionTreeClassifier(random_state=42)),
        ('rnd_cl', RandomForestClassifier(random_state=42)),
        ('kn_cl', KNeighborsClassifier())
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # number of cross-validation folds
)

stacking_clf_pipeline=make_pipeline(StandardScaler(),stacking_clf)
stacking_clf_pipeline.fit(X_train,y_train)

accuracy_voting = accuracy_score(y_test,stacking_clf_pipeline.predict(X_test))
print(f"Exactitud del ensemble (Stacking): {accuracy_voting:.4f}")

Exactitud del ensemble (Stacking): 0.9719
